# INSTALAR LIBRERIAS

In [13]:
# !pip install docx2pdf
#pip install ipywidgets

# IMPORTAR

In [1]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 7 INFORMES
7 INFORMES


['G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/40004_CRS_01-05-2025_AGUILERA RIVERO NICOLAS JULIO_475.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/41962_CRS_25-04-2025_CANDIA ROJAS JORGE LUIS_558.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.docx']

# INICIA EL PROCESO DE CONVERCIÓN

In [5]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.pdf'

In [6]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.pdf'

In [7]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\Usuario\Downloads\\' + pdf)
    i+=1
    time.sleep(5)
print('END')

CONVIRTIENDO... INFORME 1 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.pdf


...

CONVIRTIENDO... INFORME 6 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 7 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# Leer archivos

In [8]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [9]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

7 informes


{13763: '13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.pdf',
 2626: '2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.pdf',
 3412: '3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.pdf',
 40004: '40004_CRS_01-05-2025_AGUILERA RIVERO NICOLAS JULIO_475.pdf',
 41962: '41962_CRS_25-04-2025_CANDIA ROJAS JORGE LUIS_558.pdf',
 41980: '41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.pdf',
 42229: '42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.pdf'}

# CARGAR LOS PDFS A GOOGLE DRIVE

In [10]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 13763_CRS_24-04-2025_TUFINO SCHWENK RUBEN DARIO_577.pdf with mimeType application/pdf
Created file 2626_CRS_30-04-2025_CONDORI QUEVEDO JORGE_459.pdf with mimeType application/pdf
Created file 3412_CRS_30-04-2025_ESPINOZA VILLCA DIONICIO_498.pdf with mimeType application/pdf
Created file 40004_CRS_01-05-2025_AGUILERA RIVERO NICOLAS JULIO_475.pdf with mimeType application/pdf
Created file 41962_CRS_25-04-2025_CANDIA ROJAS JORGE LUIS_558.pdf with mimeType application/pdf
Created file 41980_CRS_30-04-2025_CASANOVA VELASCO JULIO CESAR_470.pdf with mimeType application/pdf
Created file 42229_CRS_29-04-2025_CANDIA ROJAS JOSE LUIS_559.pdf with mimeType application/pdf
end


In [11]:
print(len(dic_urls), 'URLs')
dic_urls

7 URLs


{13763: 'https://drive.google.com/file/d/1muGyI_sWIeX1S019tcr8swAvaqK48b1y/view?usp=drivesdk',
 2626: 'https://drive.google.com/file/d/14u6wxSQ9yIl7wdTxsa7VjOPOswNFUHjb/view?usp=drivesdk',
 3412: 'https://drive.google.com/file/d/1ZGJoMv52qT5M0MF8tVR7tqvhqPnJx4Ek/view?usp=drivesdk',
 40004: 'https://drive.google.com/file/d/1KbBcnORGcMGoSTa14L0xXVT_BVDuIoiM/view?usp=drivesdk',
 41962: 'https://drive.google.com/file/d/12ZTGhj6FScja1nVCKmLgxNwFPh_rCPvX/view?usp=drivesdk',
 41980: 'https://drive.google.com/file/d/1qlMvbVGmJqA1QCAeOZtZglwSfqxbsmMO/view?usp=drivesdk',
 42229: 'https://drive.google.com/file/d/14rZOsxu83q28N-6UUsE22RJxeZyWyitb/view?usp=drivesdk'}

# conexion a amigocloud

In [12]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [13]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [14]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

13763
2626
3412
40004
41962
41980
42229
end
